# Compressão

* Huffman

Problema: comprimir dados para diminuir espaço de armazenamento sem perda de informação

Motivação: um texto com 100k caracteres entre `a` e `f`

obs: em bits, $a=00000000$

|      |     |        |       |       |       |       |           |
|-----------------------------------------------------------------|
|      |   a  |    b  |    c  |    d  |    e  |    f  |           |
| freq |  45  |   13  |   12  |   16  |    9  |    5  | 800 kbits |
| fixo | 000  |  001  |  010  |  011  |  100  |  101  | 300 kbits |
|  var |   0  |  101  |  100  |  111  | 1101  | 1100  | 224 kbits |

`fixo` ou caractere de comprimento fixo é a representação em bits dos caracteres
`var` ou caractere de comprimento variável é o algoritmo de Huffman, o caractere mais frequente (o `a`) é assinalado menos bits (no caso, apenas um, o $0$)

Árvore que representa a compressão com tamanho fixo:

                          100
                     0/         \1
                    86           14
                 0/   \1        0/   \1
                58    28       14    null
            0/   \1  0/  \1   0/ \1
         a:45  b:13 c:12 d:16 e:9 f:5
         
         

Árvore que representa a compressão com tamanho variável:


                          100
                    0/           \1
                  a:45            55
                             0/        \1
                            25          30
                        0/    \1      0/    \1
                     c:12    b:13    14     d:16
                                   0/  \1
                                 f:5   e:9
                                 
                                 
Algortimo de Huffman da classe de algoritmos gananciosos que garante a compressão de até 90%, é estástico

```
Huffman( C ) {
    n = | C |;
    Q = C;
    for (int i=1; i <= n-1; i++) {
        z = criaNodo();
        p1 = extraiMin(Q);
        p2 = extraiMin(Q);
        esq(z) = p1;
        dir(z) = p2;
        z -> freq = p1 -> freq + p2 + freq;
        }
    insere(Q, z);
    retorna extraiMin(Q);
}
```

```
Primeiramente a fila está vazia:
Q = {null}

Depois de ler os dados e ordenar a fila:
Q = {(f:5), (e:9), (c:12), (b:13), (d:16), (a:45)}

Cria p1 e p2 ("f" e "e" respectivamente) e cria o z igual a 14:

     14
     / \
   f:5 e:9
   
Tiramos "f" e "e" da fila e inserimos a suas frequencias somadas:
Q = {(*:14), (d:16), (*:25), (a:45)}

Como árvore:

       25
     0/  \1
    c:12 b:13
                     14
                     / \
                   f:5 e:9

Próxima iterada:
Q = { (*:25), (*:30), (a:45)}



       25
     0/  \1            30
    c:12 b:13        0/   \1
                     14   d:16
                     / \
                   f:5 e:9
                   
Próxima iterada:
Q = {(a:45), (*:55)}


              55
        0/            \1
       25
     0/  \1            30
    c:12 b:13        0/   \1
                     14   d:16
                     / \
                   f:5 e:9
                   
                   
Última iterada:
Q = {(*:90)}

                   90
               0/      \1
             55         a:45

        0/         \1
       25
     0/  \1         30
    c:12 b:13     0/   \1
                 14   d:16
                / \
              f:5 e:9
                   
                  
```

Organizar o seguinte documento: "bananada de banana"

```


1.
Q = {}

2.
Q = {(e:1), (\b:2), (b:2), (d:2), (n:4), (a:7)}

3.
Q = {(b:2), (d:2), (*:3), (n:4), (a:7)}


                 3
               0/  \1
             e:1    \b:2
                             
4.
Q = {(*:3), (*:4), (n:4), (a:7)}


                 3               4
               0/  \1         0/   \1
             e:1    \b:2     b:2   d:2
             
             
5.
Q = {(n:4), (a:7), (*:7)}

                          7
                  0/            \1
                 3               4
               0/  \1         0/   \1
             e:1    \b:2     b:2   d:2
             
             
6.
Q = {(*:7), (*:11)}

                          7                     11
                  0/            \1            0/  \1
                 3               4          n:4   a:7
               0/  \1         0/   \1
             e:1    \b:2     b:2   d:2
             
             
7.
Q = {(*:18)}
                                      18
                            0/                 \1
                          7                     11
                  0/            \1            0/  \1
                 3               4          n:4   a:7
               0/  \1         0/   \1
             e:1    \b:2     b:2   d:2

```

# transação

    "Uma transação é uma sequencia de uma ou mais operacoes de acesso em um banco de dados compartilhado"
    
Exemplo: depositar R\$ 100. Ler o saldo R\$ 200, depositar R\$ 100, atualizar saldo R\$ 300

Transação T1: ler saldo (id=333), saldo = saldo + 100, escrever saldo

* Estados de uma transação:

```                                       
                                      fim das
início                               operações               commit
  das           ->        ativa         ->        parcial      ->         confirmada
operações            (ler, escrever)              /                          /
                                   \abort   abort/                          /
                                    \           /                          /
                                     \         /                          /
                                      \       /                          /
                                        falha                       terminada
```

* Propriedades de exatidão ACID:

    - Atomicidade: "tudo ou nada"
    - Consistência: "parece correto"
    - Isolamento: "como se estivesse sozinho"
    - Durabilidade: "modificações persistem após commit"
    
Mecanismo para manter **Atomicidade**: logging (caixa preta do avião)

|        Arquivo LOG       |
|--------------------------|
| INSERT id=333, saldo=200 |
| INSERT id=334, saldo=70  |
| INSERT id=335, saldo=350 |
| DELETE id=334            |
| UPDATE saldo=90 (id=335) |

|            transação           |
|--------------------------------|
| atualizar o saldo=300 (id=333) |

|        Arquivo LOG       |
|--------------------------|
| INSERT id=333, saldo=200 |
| INSERT id=334, saldo=70  |
| INSERT id=335, saldo=350 |
| DELETE id=334            |
| UPDATE saldo=90 (id=335) |
| UPDATE saldo=300 (id=333)|

**Isolamento**: transações concorrentes não se interferem

|   $T_1$   |   $T_2$   |
|-----------------------|
| transação | transação |
| transação | transação |
| transação | transação |
| transação | transação |

Problema:

|   $T_1$: depósito R\$ 100 |   $T_2$: retirada R\$100   |
|--------------------------------------------------------|
|     ler saldo (id=333)    |                            |
|        saldo = 200        |      ler saldo (id=333)    |
|    saldo = saldo + 100    |        saldo = 200         |
|      escrever saldo       |     saldo = saldo - 100    |
|                           |       escrever saldo       |

Mecanismos para manter o isolamento:
* Pessimismo: evitar que aconteça
* Otimismo: consertar o erro

Uma das maneiras é transformar transações paralelas em transações seriais

Teste de conflito de serialidade

- Criar nó para cada $T_i$ do agendamento S
- Criar aresta $T_i \arrow T_j$

Protocolos para manter a "serialidade" dos agendamentos
- Bloqueio
- MVCC (timestamp)
- Certificação

**Consistência** do banco de dados: banco de dados representa o mundo real de forma precisa e segue restrições de integridade.